In [23]:
import pandas as od
# 2. 读取数据
df = pd.read_csv('../data/processed/merged_data.csv')
df.head()

,title,play_count,date,duration,tname,tname_v2,view,reply,favorite,coin,share,like,dislike,tags,duration_seconds
0,我们最早去探明日方舟联名店但是最晚发…,581000.0,2025-5-5,13:31,搞笑,二次元线下,581916,1068,6238,10496,6399,44425,0,明日方舟 名创优品 联动,811
1,我们从网上找来了一些有趣网站…,822000.0,2025-5-3,23:08,搞笑,随拍·综合,822231,1031,15459,29698,2448,63762,0,解锁一个新兴趣 有趣 搞笑 娱乐 吐槽 电脑 把兴趣玩出名堂 互联网 网站,1388
2,谢谢你影石insta360 X5 全景相机让我们又有理由出去玩了…,307000.0,2025-5-1,15:56,搞笑,摄影摄像,307463,485,2369,5575,580,17513,0,拍照设备 影石X5 运动相机,956
3,就是因为你才没刮胡子么飞科？,486000.0,2025-4-28,03:25,搞笑,家用电器,486010,724,2034,4311,348,28489,0,飞科f8 飞科f8剃须刀 飞科f8往复式便携剃须刀,205
4,又网买玩...,1077000.0,2025-4-25,41:52,搞笑,潮玩玩具,1077579,2029,10178,25748,1938,55510,0,开箱,2512


In [27]:
# === Step 2: 解析 duration 列为秒数（支持 mm:ss）===
def parse_duration(s):
    try:
        if isinstance(s, (int, float)):
            return s  # 已经是秒数
        s = str(s).strip()
        if ':' in s:
            parts = s.split(':')
            if len(parts) == 2:
                minutes = int(parts[0])
                seconds = int(parts[1])
                return minutes * 60 + seconds
            elif len(parts) == 3:  # h:mm:ss 格式
                hours = int(parts[0])
                minutes = int(parts[1])
                seconds = int(parts[2])
                return hours * 3600 + minutes * 60 + seconds
        elif s.isdigit():
            return int(s)
    except:
        return None

df['duration'] = df['duration'].apply(parse_duration)


In [31]:
# === Step 3: 去除 duration 无效值（NaN）===
df = df.dropna(subset=['duration'])
df['duration'] = df['duration'].astype(int)
df.head()


,title,play_count,date,duration,tname,tname_v2,view,reply,favorite,coin,share,like,dislike,tags,duration_seconds
0,我们最早去探明日方舟联名店但是最晚发…,581000.0,2025-5-5,811,搞笑,二次元线下,581916,1068,6238,10496,6399,44425,0,明日方舟 名创优品 联动,811
1,我们从网上找来了一些有趣网站…,822000.0,2025-5-3,1388,搞笑,随拍·综合,822231,1031,15459,29698,2448,63762,0,解锁一个新兴趣 有趣 搞笑 娱乐 吐槽 电脑 把兴趣玩出名堂 互联网 网站,1388
2,谢谢你影石insta360 X5 全景相机让我们又有理由出去玩了…,307000.0,2025-5-1,956,搞笑,摄影摄像,307463,485,2369,5575,580,17513,0,拍照设备 影石X5 运动相机,956
3,就是因为你才没刮胡子么飞科？,486000.0,2025-4-28,205,搞笑,家用电器,486010,724,2034,4311,348,28489,0,飞科f8 飞科f8剃须刀 飞科f8往复式便携剃须刀,205
4,又网买玩...,1077000.0,2025-4-25,2512,搞笑,潮玩玩具,1077579,2029,10178,25748,1938,55510,0,开箱,2512


In [33]:
# === Step 4: 添加时长分桶列 ===
def duration_bucket(x):
    if x <= 180:
        return '0-3分钟'
    elif x <= 600:
        return '3-10分钟'
    elif x <= 1200:
        return '10-20分钟'
    else:
        return '20分钟以上'

df['duration_bucket'] = df['duration'].apply(duration_bucket)


In [37]:
# === Step 5: 添加互动率指标（防止除以 0） ===
df['view'] = pd.to_numeric(df['view'], errors='coerce').fillna(0)
df['like'] = pd.to_numeric(df['like'], errors='coerce').fillna(0)
df['coin'] = pd.to_numeric(df['coin'], errors='coerce').fillna(0)
df['favorite'] = pd.to_numeric(df['favorite'], errors='coerce').fillna(0)

df['like_rate'] = df['like'] / df['view'].replace(0, pd.NA)
df['coin_rate'] = df['coin'] / df['view'].replace(0, pd.NA)
df['favorite_rate'] = df['favorite'] / df['view'].replace(0, pd.NA)
df['interaction_rate'] = (df['like'] + df['coin'] + df['favorite']) / df['view'].replace(0, pd.NA)

# 替换无穷和NaN为0（便于Tableau读取）
df = df.fillna(0)

# === Step 6: 保存为 Tableau 可视化用的 CSV 文件 ===
output_path = "../data/processed/tableau_data.csv"
df.to_csv(output_path, index=False)
print(f"✅ 已保存处理后数据到 {output_path}，共 {len(df)} 条记录")

✅ 已保存处理后数据到 ../data/processed/tableau_data.csv，共 455 条记录


In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 455 entries, 0 to 454
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   title             455 non-null    object 
 1   play_count        455 non-null    float64
 2   date              455 non-null    object 
 3   duration          455 non-null    int64  
 4   tname             455 non-null    object 
 5   tname_v2          455 non-null    object 
 6   view              455 non-null    int64  
 7   reply             455 non-null    int64  
 8   favorite          455 non-null    int64  
 9   coin              455 non-null    int64  
 10  share             455 non-null    int64  
 11  like              455 non-null    int64  
 12  dislike           455 non-null    int64  
 13  tags              455 non-null    object 
 14  duration_seconds  455 non-null    int64  
 15  duration_bucket   455 non-null    object 
 16  like_rate         455 non-null    float64
 1